#Postprocess Real test set

In [ ]:
import pandas as pd 
import re

In [ ]:
def preprocessTest_en(df):
  '''
  drop Nan
  replace "Alright!" -> "Alright !", "Goodnight." -> "Goodnight ." เว้นจุด ! ? ที่ท้ายประโยค
  replace "Alright, bye" -> "Alright , bye"
  replace "&quot;" -> '"' (ฟันหนู)
  '''
  df = df.dropna()

  df['en_text'] = df['en_text'].map(lambda x: x[:-1] + ' .' if x.endswith('.')  else x)
  df['en_text'] = df['en_text'].map(lambda x: x[:-1] + ' ?' if x.endswith('?')  else x)
  df['en_text'] = df['en_text'].map(lambda x: x[:-1] + ' !' if x.endswith('!')  else x)

  E_idx = df[(df['en_text'].str.contains(',') == True) & (df['en_text'].str.contains(', ') == True)& (df['en_text'].str.contains(' ,') == False)].index
  df['en_text'][E_idx] = df[(df['en_text'].str.contains(',') == True) & (df['en_text'].str.contains(', ') == True)& (df['en_text'].str.contains(' ,') == False)]['en_text'].str.replace(",", " ,")

  df['en_text'].map(lambda x : re.sub('&quot;','\"',x))

  Test_df['en_post'] = Test_df['en_post'].map(lambda x : re.sub('i ','I ',x))
  Test_df['en_post'] = Test_df['en_post'].map(lambda x : re.sub("i'","I'",x))
  Test_df['en_post'] = Test_df['en_post'].map(lambda x : re.sub(" i[.]"," I.",x))
  Test_df['en_post'] = Test_df['en_post'].map(lambda x : re.sub(" i"," I",x))
  Test_df['en_post'] = Test_df['en_post'].map(lambda x : re.sub("(?<=[a-z])I","i",x))
  Test_df['en_post'] = Test_df['en_post'].map(lambda x : re.sub("I(?=[a-z])","i",x))

  return df


In [ ]:
def postprocessTest_en(df):
  '''
  hello . -> hello.
  เติม จุด ให้ถ้าไม่ใช่คำถาม
  wh-qustion to end with ?

  '''
  df = df.dropna()
  # add . 
  df['en_text'] = df['en_text'].map(lambda x: x + '.' if (x[-1] != '"' and x[-1] != '!' and x[-1] != '?' and x[-1] != '.')  else x)
  df['en_text'] = df['en_text'].str.replace(" \\.", '.')
  df['en_text'] = df['en_text'].str.replace(" \\?", '?')
  df['en_text'] = df['en_text'].str.replace(" \\!", '!')

  df['en_text'] = df['en_text'].str.replace(" 'm", "'m")
  df['en_text'] = df['en_text'].str.replace(" 's", "'s")
  df['en_text'] = df['en_text'].str.replace(" 're", "'re")
  df['en_text'] = df['en_text'].str.replace(" 'll", "'ll")
  df['en_text'] = df['en_text'].str.replace(" 't", "'t")

  # Uppercase first sentence
  df['en_text'] = df['en_text'].map(lambda x: str(x[0]).upper() + x[1:] if str(x[0]).isalpha()  else x)



  try:
    df['en_text'] = df['en_text'].map(lambda x: x[0] + str(x[1]).upper() + x[2:] if str(x[0]) == '"'  else x)
  except:
    df['en_text'] = df['en_text']

  def replace_xx(df, word, new_word, stop_word, new_stop_word):
    idx = df[df["en_text"].str.startswith(word)].index
    df.en_text[idx] = df[df['en_text'].str.startswith(word)]['en_text'].str.replace(word, new_word)
    idxx = df[df["en_text"].str.endswith(stop_word) & df["en_text"].str.startswith(new_word)].index
    df['en_text'][idxx] = df['en_text'][idxx].map(lambda x: x[:-1] + new_stop_word if True  else x)
    return df  

  replace_xx(df,"do ", "Do ", ".", "?")
  replace_xx(df,"did ", "Did ", ".", "?")
  replace_xx(df,"does ", "Does ", ".", "?")
  replace_xx(df,"what ", "What ", ".", "?")
  replace_xx(df,"when ", "When ", ".", "?")
  replace_xx(df,"where ", "Where ", ".", "?")
  replace_xx(df,"why ", "Why ", ".", "?")
  replace_xx(df,"how ", "How ", ".", "?")
  replace_xx(df,"can ", "Can ", ".", "?")
  replace_xx(df,"could ", "Could ", ".", "?")
  replace_xx(df,"may ", "May ", ".", "?")
  replace_xx(df,"might ", "Might ", ".", "?")

  # startswith "i " -> "I "
  idx = df[df["en_text"].str.startswith('i ')].index
  df.en_text[idx] = df['en_text'][idx].str.replace('i ', 'I ')
  # " i " -> " I "
  df['en_text'].str.replace(' i ', ' I ')

  df['en_text'] = df['en_text'].map(lambda x : re.sub('i ','I ',x))
  df['en_text'] = df['en_text'].map(lambda x : re.sub("i'","I'",x))
  df['en_text'] = df['en_text'].map(lambda x : re.sub(" i[.]"," I.",x))
  df['en_text'] = df['en_text'].map(lambda x : re.sub(" i"," I",x))
  df['en_text'] = df['en_text'].map(lambda x : re.sub("(?<=[a-z])I","i",x))
  df['en_text'] = df['en_text'].map(lambda x : re.sub("I(?=[a-z])","i",x))

  df.en_text = df['en_text'].str.replace('\\.\\?', '?')
  df.en_text = df['en_text'].str.replace('\\?\\?', '?')
  df.en_text = df['en_text'].str.replace('\\.\\.\\.\\.', '<unk>')
  df.en_text = df['en_text'].str.replace('\\.\\.', '.')
  df.en_text = df['en_text'].str.replace('<unk>', '...')
  df.en_text = df['en_text'].str.replace('tHe', 'the')
  df.en_text = df['en_text'].str.replace('"\\.', '"')
  df.en_text = df['en_text'].str.replace('เฟอร์', '')
  df.en_text = df['en_text'].str.replace(' ลาย', '')
  df.en_text = df['en_text'].str.replace(' จาง ', ' ')

  df['en_text'] = df['en_text'].map(lambda x: str(x[0]).upper() + x[1:] if str(x[0]).isalpha()  else x)
  
   
  return df


In [ ]:
def processTest_th(df):
  # --------------ถ้า preprocess ก่อน predict ให้รันทั้งหมด -----------------------
  df = df.fillna("")
  def removeunderscore(df): 
    df['th_text'] = df['th_text'].str.replace("_", "") 
    return df
  df = removeunderscore(df)

  def fill_th_q(df, word, new_word):
    idx = df[df["th_text"].str.endswith(word)].index
    df['th_text'][idx] = df[df['th_text'].str.endswith(word)]["th_text"].str.replace(word, new_word)
    return df

  fill_th_q(df, " ไหม", " ไหม ?")
  fill_th_q(df, " อะไร", " อะไร ?")
  fill_th_q(df, " รึยัง", " รึยัง ?")
  fill_th_q(df, " อย่าง ไร ดี", " อย่าง ไร ดี ?")
  fill_th_q(df, "กี่โมง", "กี่โมง ?")

  def dup_th(df, word, new_word):
    idx = df[df["th_text"].str.contains(word)].index
    df.th_text[idx] = df[df['th_text'].str.contains(word)]["th_text"].str.replace(word, new_word)
    return df

  dup_th(df, "มาก มาก ", "มาก ๆ ")
  dup_th(df, "ทุก ทุก ", "ทุก ๆ ")
  dup_th(df, "แน่ แน่ ", "แน่ ๆ ")
  dup_th(df, "อื่น อื่น ", "อื่น ๆ ")
  dup_th(df, "ค่อย ค่อย ", "ค่อย ๆ ")
  dup_th(df, "คร่าว คร่าว ", "คร่าว ๆ ")
  dup_th(df, "จิ๊บ จิ๊บ ", "จิ๊บ ๆ ")
  dup_th(df, "จริง จริง ", "จริง ๆ ")
  dup_th(df, "ข้าง ข้าง ", "ข้าง ๆ ")

  # ------------ถ้าทำ postprocess ต่อจากการ predict ให้คอมเม้นทั้งชุดนี้ออกไป-------------
  # def removeunderscore(df): 
  #   df['th_Sep'] = df['th_Sep'].str.replace("_", "") 
  #   return df
  # df = removeunderscore(df)

  # def fill_th_q(df, word, new_word):
  #   idx = df[df["th_Sep"].str.endswith(word)].index
  #   df['th_Sep'][idx] = df[df['th_Sep'].str.endswith(word)]["th_Sep"].str.replace(word, new_word)
  #   return df

  # fill_th_q(df, " ไหม", " ไหม ?")
  # fill_th_q(df, " อะไร", " อะไร ?")
  # fill_th_q(df, " รึยัง", " รึยัง ?")
  # fill_th_q(df, " อย่าง ไร ดี", " อย่าง ไร ดี ?")
  # fill_th_q(df, "กี่โมง", "กี่โมง ?")

  # def dup_th(df, word, new_word):
  #   idx = df[df["th_Sep"].str.contains(word)].index
  #   df.th_Sep[idx] = df[df['th_Sep'].str.contains(word)]["th_Sep"].str.replace(word, new_word)
  #   return df

  # dup_th(df, "มาก มาก ", "มาก ๆ ")
  # dup_th(df, "ทุก ทุก ", "ทุก ๆ ")
  # dup_th(df, "แน่ แน่ ", "แน่ ๆ ")
  # dup_th(df, "อื่น อื่น ", "อื่น ๆ ")
  # dup_th(df, "ค่อย ค่อย ", "ค่อย ๆ ")
  # dup_th(df, "คร่าว คร่าว ", "คร่าว ๆ ")
  # dup_th(df, "จิ๊บ จิ๊บ ", "จิ๊บ ๆ ")
  # dup_th(df, "จริง จริง ", "จริง ๆ ")
  # dup_th(df, "ข้าง ข้าง ", "ข้าง ๆ ")

  # df.drop(columns=['th_sent'])
  # df['th_sent'] = df['th_Sep'].str.replace(" ", "")
  # -------------------------------------------------------------

  return df